<a href="https://colab.research.google.com/github/FARTASH-421/AI2/blob/master/Project_CR1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [208]:
import random

#Part A

In [131]:
def mul(num1, num2):
  bin_to_Int = int (num1, 2) *  int(num2, 2)
  result = bin_to_Int % ((2 ** 16) + 1)
  lis = format(result, 'b')
  l = len(lis)
  len_zero = 16 - l
  zero_front = ''.join([str(0) for i in range(len_zero)])
  return zero_front+lis


In [132]:
def sum(num1, num2):
  bin_to_Int = int (num1, 2) +  int(num2, 2)
  result = bin_to_Int % (2 ** 16)
  lis = format(result, 'b')
  l = len(lis)
  len_zero = 16 - l
  zero_front = ''.join([str(0) for i in range(len_zero)])
  return zero_front+lis


In [159]:
def xor(lis_1, lis_2, size):
  int_x = int(lis_1, 2)
  int_y = int(lis_2, 2)
  result = int_x ^ int_y
  str_result = format(result, 'b')
  l = len(str_result)
  len_zero = size - l
  zero_front = ''.join([str(0) for i in range(len_zero)])
  return zero_front + str_result

In [160]:
def Round (list_P, list_K):
    P1 = list_P[0 :16]
    P2 = list_P[16:32]
    P3 = list_P[32:48]
    P4 = list_P[48:64]

    # print(f'P1: size:{len(P1)} is-> {P1}')
    # print(f'P2: size:{len(P2)} is-> {P2}')
    # print(f'P3: size:{len(P3)} is-> {P3}')
    # print(f'P4: size:{len(P4)} is-> {P4}')

    step_1 = mul (P1, list_K[0])
    step_2 = sum (P2, list_K[1])
    step_3 = sum (P3, list_K[2])
    step_4 = sum (P4, list_K[3])
    step_5 = xor (step_1, step_3, 16)
    step_6 = xor (step_2, step_4, 16)
    step_7 = mul (step_5, list_K[4])
    step_8 = sum (step_6, step_7)
    step_9 = mul (step_8, list_K[5])
    step_10 = sum (step_7, step_9)

    step_11 = xor (step_1, step_9, 16)
    step_12 = xor (step_3, step_9, 16)
    step_13 = xor (step_2, step_10, 16)
    step_14 = xor (step_4, step_10, 16)
    return step_11 + step_12 + step_13 + step_14



In [168]:
def Block_Cipher(plaintext, key):
  index_key = 0
  for i in range(8):
    part_key = key[index_key:(index_key+6)]
    plaintext = Round(plaintext, key)
    index_key += 6

  R_1 = plaintext[0:16]
  R_2 = plaintext[16:32]
  R_3 = plaintext[32:48]
  R_4 = plaintext[48:64]


  step_1 = mul (R_1, key[index_key])
  step_2 = sum (R_2, key[index_key+1])
  step_3 = sum (R_3, key[index_key+2])
  step_4 = mul (R_4, key[index_key+3])

  ciphertext = step_1 + step_2 +step_3 + step_4
  return ciphertext


In [169]:
# ...... Test def Block_Cryptography .............

ls = '1010100000000000111000001111100000001111001010101010110101010111'
key = '1000011101010101010101011011001010110101010111010100'
print(len(key))

ciphertext = Block_Cipher(ls, key)
print(len(ciphertext))
ciphertext

52
64


'0000000000000000111000001111100100001111001010100000000000000000'

#Part B


In [206]:
def CBC(plaintext, key):

  cipher_text = ''
  bin_plaintext = ''.join([format(ord(char),'b') for char in plaintext])
  bin_key = ''.join([format(ord(char),'b') for char in key])

  len_ciphertext = len(bin_plaintext)
  len_key = len(bin_key)


  len_zero = 52 - len_key
  zero_front = ''.join([str(0) for i in range(len_zero)])
  bin_key = zero_front + bin_key

  random_num = int((random.random()*100000) % (2 ** 16))
  bin_rand = format(random_num, 'b')

  for i in range(0,len_ciphertext, 64):
    if((i < len_ciphertext) and ((i+64) > len_ciphertext)):
      index_temp = len_ciphertext - i
      plain_tt = bin_plaintext[i : i+index_temp]
      out_xor = xor(plain_tt, bin_rand, 64)
      bin_rand = Block_Cipher(out_xor, bin_key)
      cipher_text += bin_rand

    else:
      plain_t = bin_plaintext[i:(i+64)]
      out_xor = xor(plain_t, bin_rand, 64)
      bin_rand = Block_Cipher(out_xor, bin_key)
      cipher_text += bin_rand

  print(cipher_text)
  results = ''.join([chr((int(cipher_text[i:i+7], 2)%26) + 65) for i in range(0, len(cipher_text), 8)])
  print(results)
  return results



In [183]:
text = "FartashGoToSchool"
key = "jonn"
x = CBC(text, key)



00000000000000001001011101001101001111001111010100000000000000000000000000000000001010011101010000000001111011110000000000000000
AAXMESAAAAUCAPAA


#Part C


In [194]:
def UpDataCounterBin(counter):
  counter = counter + 1
  bin_counter = format(counter, 'b')
  zero = ''.join([str(0) for i in range(64 - len(bin_counter))])
  return zero + bin_counter


In [204]:
def CTR(plaintext, key):

  cipher_text = ''
  bin_plaintext = ''.join([format(ord(char),'b') for char in plaintext])
  bin_key = ''.join([format(ord(char),'b') for char in key])

  len_ciphertext = len(bin_plaintext)
  len_key = len(bin_key)

  bin_counter = '0'
  len_zero = 52 - len_key
  zero_front = ''.join([str(0) for i in range(len_zero)])
  bin_key = zero_front + bin_key


  for i in range(0,len_ciphertext, 64):

    bin_counter = UpDataCounterBin(int(bin_counter, 2))

    if((i < len_ciphertext) and ((i+64) > len_ciphertext)):

      out_EncryptBlock = Block_Cipher(bin_counter, bin_key)
      index_temp = len_ciphertext - i
      plain_tt = bin_plaintext[i : i+index_temp]
      out_xor = xor(plain_tt, out_EncryptBlock, 64)
      cipher_text += out_xor

    else:
      out_EncryptBlock = Block_Cipher(bin_counter, bin_key)
      plain_t = bin_plaintext[i:(i+64)]
      out_xor = xor(plain_t, out_EncryptBlock, 64)
      cipher_text += out_xor

  print(cipher_text)
  results = ''.join([chr((int(cipher_text[i:i+7], 2)%26) + 65) for i in range(0, len(cipher_text), 8)])
  print(results)
  return results



In [205]:
text = "FartashGoToSchool"
key = "jonn"
x = CTR(text, key)

10001101100001111001011101001101001111001111010101000111110111110000000000101001101111101001111100111101000110101111011111101100
SPXMESJHAURBENTO
